# Customer Churn Prediction with Amazon SageMaker Autopilot
_**Using AutoPilot to Predict Mobile Customer Departure**_

---

---


## Introduction

Amazon SageMaker Autopilot is an automated machine learning (commonly referred to as AutoML) solution for tabular datasets. You can use SageMaker Autopilot in different ways: on autopilot (hence the name) or with human guidance, without code through SageMaker Studio, or using the AWS SDKs. This notebook, as a first glimpse, will use the AWS SDKs to simply create and deploy a machine learning model.

Losing customers is costly for any business.  Identifying unhappy customers early on gives you a chance to offer them incentives to stay.  This notebook describes using machine learning (ML) for the automated identification of unhappy customers, also known as customer churn prediction. ML models rarely give perfect predictions though, so this notebook is also about how to incorporate the relative costs of prediction mistakes when determining the financial outcome of using ML.

We use an example of churn that is familiar to all of us–leaving a mobile phone operator.  Seems like I can always find fault with my provider du jour! And if my provider knows that I’m thinking of leaving, it can offer timely incentives–I can always use a phone upgrade or perhaps have a new feature activated–and I might just stick around. Incentives are often much more cost effective than losing and reacquiring a customer.



### Reticulating the Amazon SageMaker Python SDK

First, load the `reticulate` library and import the `sagemaker` Python module. Once the module is loaded, use the `$` notation in R instead of the `.` notation in Python to use available classes. 

In [ ]:
# Turn warnings off globally
options(warn=-1)

In [ ]:
# Install reticulate library and import sagemaker
library(reticulate)
sagemaker <- import('sagemaker')

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
session <- sagemaker$Session()
bucket <- session$default_bucket()
prefix <- 'data/r-churn'
role_arn <- sagemaker$get_execution_role()

bucket
role_arn

---
## Data

Mobile operators have historical records on which customers ultimately ended up churning and which continued using the service. We can use this historical information to construct an ML model of one mobile operator’s churn using a process called training. After training the model, we can pass the profile information of an arbitrary customer (the same profile information that we used to train the model) to the model, and have the model predict whether this customer is going to churn. Of course, we expect the model to make mistakes–after all, predicting the future is tricky business! But I’ll also show how to deal with prediction errors.

The dataset we will use is synthetically generated, but indictive of the types of features you'd see in this use case.

In [ ]:
session$download_data(path = './', 
                      bucket = 'sagemaker-sample-files', 
                      key_prefix = 'datasets/tabular/synthetic/churn.txt')

Before you run Autopilot on the dataset, first perform a check of the dataset to make sure that it has no obvious errors. The Autopilot process can take long time, and it's generally a good practice to inspect the dataset before you start a job. This particular dataset is small, so you can inspect it in the notebook instance itself. If you have a larger dataset that will not fit in a notebook instance memory, inspect the dataset offline using a big data analytics tool like Apache Spark. [Deequ](https://github.com/awslabs/deequ) is a library built on top of Apache Spark that can be helpful for performing checks on large datasets. 

Read the data into a data frame and take a look.

In [ ]:
library(readr)

churn <- read_csv(file = 'churn.txt')
head(churn)

By modern standards, it’s a relatively small dataset, with only 5,000 records, where each record uses 21 attributes to describe the profile of a customer of an unknown US mobile operator. The attributes are:

- `State`: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- `Account Length`: the number of days that this account has been active
- `Area Code`: the three-digit area code of the corresponding customer’s phone number
- `Phone`: the remaining seven-digit phone number
- `Int’l Plan`: whether the customer has an international calling plan: yes/no
- `VMail Plan`: whether the customer has a voice mail feature: yes/no
- `VMail Message`: presumably the average number of voice mail messages per month
- `Day Mins`: the total number of calling minutes used during the day
- `Day Calls`: the total number of calls placed during the day
- `Day Charge`: the billed cost of daytime calls
- `Eve Mins, Eve Calls, Eve Charge`: the billed cost for calls placed during the evening
- `Night Mins`, `Night Calls`, `Night Charge`: the billed cost for calls placed during nighttime
- `Intl Mins`, `Intl Calls`, `Intl Charge`: the billed cost for international calls
- `CustServ Calls`: the number of calls placed to Customer Service
- `Churn?`: whether the customer left the service: true/false

The last attribute, `Churn?`, is known as the target attribute–the attribute that we want the ML model to predict.

### Upload the dataset to S3

Now we'll upload the data to a S3 bucket in our own AWS account so Autopilot can access it.

In [ ]:
write_csv(churn, 'churn.csv', col_names = TRUE)

In [ ]:
s3_train <- session$upload_data(path = 'churn.csv', 
                                bucket = bucket, 
                                key_prefix = prefix)

s3_train

---
## Launching a SageMaker Autopilot Job<a name="Settingup"></a>

After uploading the dataset to Amazon S3, you can launch Autopilot to find the best ML pipeline to train a model on this dataset. 

Currently Autopilot supports only tabular datasets in CSV format. Either all files should have a header row, or the first file of the dataset, when sorted in alphabetical/lexical order by name, is expected to have a header row.

We'll launch an Autopilot job via the Studio UI (it also is possible to do so via API).  To do so:

- Go to the tilted triangle icon in the left toolbar and click it, then select **Experiments and trials**.
- Click the **Create Autopilot Experiment** button.
- For **Experiment name**, enter a name such as `automl-churn-` with a date suffix. e.g. `automl-churn-10-14-21`
- Skip to **CONNECT YOUR DATA**, then find the **S3 bucket name** using autocomplete by typing `sagemaker-` and matching to the bucket name printed below the previous code cell.  Similarly, find the **Dataset file name** the same way, it should be `data/r-churn/churn.csv`
- Skip to **Target**, and select `Churn?` from the drop down menu.
- Skip to **Output data location**, select the radio button for **Enter S3 bucket location**, and then enter a string such as `s3://<your-bucket-name>/data/r-churn/output` where you replace `your-bucket-name` with the bucket name you've used previously.
- Go to **Auto deploy** and switch it to off.  
- Click the down arrow for **Advanced Settings**, go to **Max candidates** and enter 20.  (This is to keep the runtime of the job within reasonable limits for a workshop setting.)  
- Click **Create Experiment**.

### Tracking SageMaker Autopilot job progress<a name="Tracking"></a>
SageMaker Autopilot job consists of the following high-level steps : 

* Analyzing Data, where the dataset is analyzed and Autopilot comes up with a list of ML pipelines that should be tried out on the dataset. The dataset is also split into train and validation sets.
* Feature Engineering, where Autopilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
* Model Tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline). 

Although we can use code/API calls to track job progress, we'll use the SageMaker Studio UI to do so.  After you create the job via the Studio UI above, the tab will convert to an Autopilot Job tracking tab.   You'll be able to see the progress of the job in that tab.

If you close the tab you can always get back to it.  To do so, go to the tilted triangle icon in the left toolbar and click it, then select **Experiments and trials**.  Next, right-click the name of your AutoML job, which should start with "automl-churn-", and select **Describe AutoML Job**.  A new Studio tab will open details about your job, and a summary when it completes, with the ability to sort models by metric and deploy with a single click.  

### Model Explainability

Autopilot also generates an explainability report using SageMaker Clarify.  The report and related artifacts are uploaded to S3, but you also can access the report in SageMaker Studio.

To do so:
- Go to the tilted triangle icon in the left toolbar and click it, then select **Experiments and trials**.
- In the list of experiments, click on ***Unassigned trial components***.
- Double-click the trial component with the name of the form, `automl-churn-<data-suffix>-documentation`.
- A new tab will open named `Describe Trial Component`; in it you will see a graph of feature importance by aggregated SHAP values.  Of the 20 original input features, Clarify plots the 10 features with the greatest feature attribution.

Clarify uses a model-agnostic feature attribution approach, which you can used to understand why a model made a prediction after training and to provide per-instance explanation during inference. The implementation includes a scalable and efficient implementation of SHAP.

It also is possible to visualize the the local explanations for single examples in your dataset. You can simply load the local explanations stored in the Autopilot explainability output path, and visualize the explanation (i.e., the impact that the single features have on the prediction of your model) for any example.  Typically for an example you would plot a bar chart with SHAP values for each feature.  The larger the bar, the more impact the feature has on the target feature. Bars with positive values are associated with higher predictions in the target variable, and bars with negative values are associated with lower predictions in the target variable.